In [42]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=8, app_name='west0_ws_seg', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_ws_seg...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:27


In [94]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [44]:
# 각 day 30
start_date = "2022-06-13"
end_date = "2022-07-12"

ws_start_date = "2022-07-13"
ws_day_30 = "2022-08-11"

In [4]:
pre_gcoin = load_data_mart("pc", start_date, end_date, "gcoin_use")

In [5]:
wsus_gcoin = pre_gcoin.where(col("event_name").like("%wsus%"))
wsus_user = wsus_gcoin.select("account_id").distinct()

In [6]:
costume = pre_gcoin.where((col("sub_category").isin(["costume", "gear", "vehicle"])) | \
                          ((col("sub_category") == "crate") & (~col("event_name").like("%wsus%")) & (~col("event_name").like("%survivorpass%"))))
costume_user = costume.select("account_id").distinct()

In [7]:
costume.groupBy("sub_category", "category", "event_type").agg(count("*")).show(truncate=False)

+------------+-----------+-------------+--------+
|sub_category|category   |event_type   |count(1)|
+------------+-----------+-------------+--------+
|costume     |skin       |other        |29471   |
|gear        |skin       |other        |54510   |
|costume     |skin       |yourshop     |2308    |
|crate       |crates&keys|collaboration|45042   |
|costume     |skin       |streamer     |56330   |
|vehicle     |skin       |other        |769     |
|crate       |crates&keys|other        |148543  |
|crate       |crates&keys|esports      |8316    |
|crate       |crates&keys|streamer     |82445   |
|crate       |crates&keys|other        |11856   |
|gear        |skin       |yourshop     |218     |
|vehicle     |skin       |yourshop     |165     |
|costume     |skin       |other        |183339  |
|crate       |crates&keys|yourshop     |1788    |
+------------+-----------+-------------+--------+



In [8]:
only_costume_gcoin = costume.join(wsus_user, "account_id", "leftanti")
costume_and_wsus_gcoin = wsus_gcoin.join(costume_user, "account_id")
only_wsus_gcoin= wsus_gcoin.join(costume_user, "account_id", "leftanti")

In [9]:
only_costume_gcoin.groupBy("sub_category", "category", "event_type").agg(count("*")).show(truncate=False)

+------------+-----------+-------------+--------+
|sub_category|category   |event_type   |count(1)|
+------------+-----------+-------------+--------+
|costume     |skin       |other        |24362   |
|gear        |skin       |other        |42475   |
|costume     |skin       |yourshop     |1535    |
|crate       |crates&keys|collaboration|28572   |
|costume     |skin       |streamer     |43610   |
|vehicle     |skin       |other        |477     |
|crate       |crates&keys|other        |106749  |
|crate       |crates&keys|esports      |4708    |
|crate       |crates&keys|streamer     |52612   |
|crate       |crates&keys|other        |9158    |
|gear        |skin       |yourshop     |147     |
|vehicle     |skin       |yourshop     |93      |
|costume     |skin       |other        |140324  |
|crate       |crates&keys|yourshop     |1032    |
+------------+-----------+-------------+--------+



In [10]:
only_wsus_gcoin.groupBy("sub_category", "category", "event_type").agg(count("*")).show(truncate=False)

+------------+-----------+----------+--------+
|sub_category|category   |event_type|count(1)|
+------------+-----------+----------+--------+
|crate       |crates&keys|wsus      |1539589 |
+------------+-----------+----------+--------+



- a) only_costume_gcoin
- b) costume_and_wsus_gcoin
- c) only_wsus_gcoin

In [45]:
def get_user_info(end_date):
    def classify_country(country_os, country_ip):
        if country_os != 'CN':
            return country_ip
        else:
            return country_os

    country_type_udf = udf(classify_country, StringType())
    meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

    user = load_data_mart("pc", end_date, end_date, "user_master")
    user = user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
    user = user.join(meta_region, user.country_new == meta_region.country_code_iso2, "left").withColumnRenamed("accountid", "AccountId").select("AccountId", "pubg_region", "lastlogindate")
    return user

In [46]:
def get_workshop_rate(start_date, end_date):
    user = get_user_info(end_date)
    
    bp_purchase = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where((col("Currency").like("%bp%")) & (col("AnalyticEventType") == "workshop")) \
        .join(user, "AccountId")
    gcoin = load_data_mart("pc", start_date, end_date, "gcoin_use").where(col("event_name").like("%workshop%")).withColumnRenamed("account_id", "AccountId") \
        .join(user, "AccountId")
    open_crate = load_schema.lobby(spark, "pc", "live", "WorkshopCrateOpened", start_date, end_date) \
        .join(user, "AccountId")
    craft = load_schema.lobby(spark, "pc", "live", "WorkshopCrafted", start_date, end_date) \
        .join(user, "AccountId")
    disassemble = load_schema.lobby(spark, "pc", "live", "WorkshopDisassembled", start_date, end_date) \
        .join(user, "AccountId")
    repurpose = load_schema.lobby(spark, "pc", "live", "WorkshopRepurposed", start_date, end_date) \
        .join(user, "AccountId")
    special_craft = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where(col("Currency") == "artisanstoken") \
        .join(user, "AccountId")
    
    workshop_user = (open_crate.groupBy("pubg_region", "AccountId").agg(sum("OpenAmount").alias("amount")).withColumn("action", lit("open"))) \
        .unionByName(craft.groupBy("pubg_region", "AccountId").agg(count("*").alias("amount")).withColumn("action", lit("craft"))) \
        .unionByName(disassemble.groupBy("pubg_region", "AccountId").agg(count("*").alias("amount")).withColumn("action", lit("disassemble"))) \
        .unionByName(repurpose.groupBy("pubg_region", "AccountId").agg(count("*").alias("amount")).withColumn("action", lit("repurpose"))) \
        .unionByName(special_craft.groupBy("pubg_region", "AccountId").agg(sum("Amount").alias("amount")).withColumn("action", lit("special_craft"))) \
        .unionByName(gcoin.groupBy("pubg_region", "AccountId").agg(sum("qty").alias("amount")).withColumn("action", lit("gcoin"))) \
        .unionByName(bp_purchase.groupBy("pubg_region", "AccountId").agg(count("*").alias("amount")).withColumn("action", lit("bp_workshop")))
    
    return workshop_user

In [73]:
user = get_user_info(ws_day_30)

In [79]:
# AU for Day 30 after workshop released
get_user_info(ws_day_30).where(col('lastlogindate') >= ws_start_date).select(countDistinct("AccountId").alias("au")).show()

+-------+
|     au|
+-------+
|9975867|
+-------+



In [85]:
ws_user = get_workshop_rate(ws_start_date, ws_day_30)
ws_user.select(countDistinct("AccountId").alias("ws user cnt")).show()
ws_user.groupBy("action").agg(countDistinct("AccountId").alias("user_cnt")).toPandas()

+-----------+
|ws user cnt|
+-----------+
|    2904880|
+-----------+



,action,user_cnt
0,disassemble,1262284
1,repurpose,251644
2,bp_workshop,2039304
3,craft,1373360
4,gcoin,703276
5,special_craft,287103
6,open,2733723


In [47]:
ws_user = get_workshop_rate(ws_start_date, ws_day_30)

In [13]:
from pyspark.sql.types import * 

def wsus_cnt(price):
    if price == 1800:
        return 10
    elif price == 200:
        return 1
    else:
        return 0
    
assign_wsus_cnt = udf(wsus_cnt, IntegerType())

In [60]:
ws_gcoin = load_data_mart("pc", ws_start_date, ws_day_30, "gcoin_use").where(col("event_name").like("%workshop%"))

In [14]:
gcoin_by_type = load_data_mart("pc", ws_start_date, ws_day_30, "gcoin_use") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop") \
                .when((col("sub_category").isin(["costume", "gear", "vehicle"])) | \
                          ((col("sub_category") == "crate") & (~col("event_name").like("%wsus%")) & (~col("event_name").like("%workshop%")) & (~col("event_name").like("%survivorpass%"))), "costume") \
                .when(col("event_name").like("%wsus%"), "wsus")) \
    .where(col("type").isin(["workshop", "costume", "wsus"])) \
    .withColumn("wsus_cnt", when(col("type") == "wsus", assign_wsus_cnt("price")).otherwise(lit(None)))

In [15]:
pre_gcoin_by_type = load_data_mart("pc", start_date, end_date, "gcoin_use") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop") \
                .when((col("sub_category").isin(["costume", "gear", "vehicle"])) | \
                          ((col("sub_category") == "crate") & (~col("event_name").like("%wsus%")) & (~col("event_name").like("%workshop%")) & (~col("event_name").like("%survivorpass%"))), "costume") \
                .when(col("event_name").like("%wsus%"), "wsus")) \
    .where(col("type").isin(["workshop", "costume", "wsus"])) \
    .withColumn("wsus_cnt", when(col("type") == "wsus", assign_wsus_cnt("price")).otherwise(lit(None)))

In [16]:
pre_gcoin_by_type.join(only_costume_gcoin.select("account_id").distinct(), "account_id").groupBy("type").agg(countDistinct("account_id")).show(truncate=False)

+-------+--------------------------+
|type   |count(DISTINCT account_id)|
+-------+--------------------------+
|costume|323990                    |
+-------+--------------------------+



## a 그룹

In [17]:
only_costume_gcoin_by_user = only_costume_gcoin.groupBy("account_id").agg(sum(col("free_use") + col("paid_use")).alias("total_gcoin"), sum("paid_use").alias("paid_use"))

In [18]:
only_costume_gcoin_by_user = only_costume_gcoin_by_user.withColumn("group", \
                                         when(col("total_gcoin") <= 1000, "A") \
                                        .when((col("total_gcoin") > 1000) & (col("total_gcoin") <= 5000), "B") \
                                        .when((col("total_gcoin") > 5000) & (col("total_gcoin") <= 10000), "C") \
                                        .when((col("total_gcoin") > 10000) & (col("total_gcoin") <= 30000), "D") \
                                        .when((col("total_gcoin") > 30000), "E") \
                                     )
only_costume_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_gcoin").alias("total_gcoin"), sum("paid_use").alias("paid_use")).orderBy("group").toPandas()

,group,user_cnt,total_gcoin,paid_use
0,A,206637,112139030,14442175
1,B,115117,201177700,65289545
2,C,2127,13714530,11242400
3,D,107,1342230,1110560
4,E,2,73260,64380


In [19]:
only_costume_gcoin_by_user.where(col("paid_use")>0).groupBy("group").agg(countDistinct("account_id").alias("paid_pu")).orderBy("group").toPandas()

,group,paid_pu
0,A,37998
1,B,47528
2,C,2088
3,D,104
4,E,2


In [20]:
only_costume_gcoin_by_user.toPandas().to_csv("./only_costume_gcoin_by_user.csv", index=False)

In [80]:
only_costume_gcoin_by_user = pd.read_csv("./only_costume_gcoin_by_user.csv")
only_costume_gcoin_by_user.head()

,account_id,total_gcoin,paid_use,group
0,account.001a84b7c1f44ad6896aacd5f0e68589,300,0,A
1,account.003b956ea1a842188a565d944c174454,300,0,A
2,account.00593012c1c44c319d59b13a0e870ee0,1000,0,A
3,account.00bb345b36ec405a99bf6c938cf16874,200,0,A
4,account.00d1973f6a6d4ccaa8614db57402d9b2,500,0,A


In [79]:
# AU by group
user.withColumnRenamed("AccountId", "account_id").where(col("lastlogindate") >= ws_start_date).join(only_costume_gcoin_by_user, "account_id").groupBy("group").agg(countDistinct("account_id").alias("au")).orderBy("group").toPandas()

,group,au
0,A,144488
1,B,80112
2,C,1914
3,D,99
4,E,2


In [38]:
only_costume_gcoin_by_user = spark.createDataFrame(only_costume_gcoin_by_user)
workshop_a = ws_user.join(only_costume_gcoin_by_user.withColumnRenamed("account_id", "AccountId"), "AccountId")
workshop_a.groupBy("group", "action").agg(countDistinct("AccountId").alias("user_cnt")).orderBy("group", "action").toPandas()

,group,action,user_cnt
0,A,bp_workshop,74740
1,A,craft,59539
2,A,disassemble,56946
3,A,gcoin,35487
4,A,open,92684
5,A,repurpose,14929
6,A,special_craft,16912
7,B,bp_workshop,44309
8,B,craft,35175
9,B,disassemble,33641


In [39]:
workshop_a.groupBy("group").agg(countDistinct("AccountId").alias("ws user cnt")).orderBy("group").toPandas()

,group,ws user cnt
0,A,96035
1,B,54468
2,C,1663
3,D,86
4,E,1


In [22]:
pre_gcoin_by_type.join(only_costume_gcoin_by_user.select("account_id", "group"), "account_id") \
    .groupBy("account_id", "group", "type").agg(sum("paid_use").alias("paid_use"), sum("free_use").alias("free_use"), sum("wsus_cnt").alias("wsus_cnt")) \
    .withColumn("paid_user", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("total_pu"), countDistinct("paid_user").alias("paid_pu"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("wsus_cnt").alias("wsus_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,total_pu,paid_pu,paid_use,total_use,wsus_cnt
0,A,costume,206637,37998,14442175,112139030,None
1,B,costume,115117,47528,65289545,201177700,None
2,C,costume,2127,2088,11242400,13714530,None
3,D,costume,107,104,1110560,1342230,None
4,E,costume,2,2,64380,73260,None


In [64]:
group_a_workshop_gcoin_by_product = ws_gcoin.join(only_costume_gcoin_by_user.select("account_id", "group"), "account_id").groupBy("group", "product_name") \
    .agg(countDistinct("account_id").alias("pu"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).toPandas()
group_a_workshop_gcoin_by_product.to_csv("./group_a_workshop_gcoin_by_product.csv", index=False)

In [82]:
group_a_workshop_gcoin_by_product.pivot(index="product_name", columns="group", values="pu")

group,A,B,C,D,E
product_name,,,,,
ARCHIVIST'S CHEST,7503.0,4938.0,270.0,16.0,1.0
ARCHIVIST'S CHEST BUNDLE (x1),19440.0,13883.0,685.0,38.0,1.0
ARCHIVIST'S CHEST BUNDLE (x10),6778.0,7910.0,784.0,53.0,1.0
ARCHIVIST'S CHEST BUNDLE (x5),6068.0,5915.0,457.0,34.0,NaN
HUNTER'S CHEST,11060.0,6290.0,298.0,18.0,1.0
HUNTER'S CHEST PACK (x1),7409.0,5223.0,309.0,13.0,NaN
HUNTER'S CHEST PACK (x5),3852.0,3489.0,344.0,24.0,1.0
KEY,16451.0,11918.0,749.0,48.0,1.0


In [70]:
group_a_workshop_gcoin_by_product.sort_values(["group", "product_name"])

,group,product_name,pu,paid_use,total_use
29,A,ARCHIVIST'S CHEST,7503,1149490,2568240
4,A,ARCHIVIST'S CHEST BUNDLE (x1),19440,7627105,14758080
26,A,ARCHIVIST'S CHEST BUNDLE (x10),6778,70238550,73869600
24,A,ARCHIVIST'S CHEST BUNDLE (x5),6068,9372800,12956400
31,A,HUNTER'S CHEST,11060,868055,2453640
30,A,HUNTER'S CHEST PACK (x1),7409,2616600,4651960
6,A,HUNTER'S CHEST PACK (x5),3852,23800280,26361550
10,A,KEY,16451,4536270,8494600
2,B,ARCHIVIST'S CHEST,4938,1068765,1975680
15,B,ARCHIVIST'S CHEST BUNDLE (x1),13883,6728415,12285120


In [23]:
gcoin_by_type.join(only_costume_gcoin_by_user.select("account_id", "group"), "account_id") \
    .groupBy("account_id", "group", "type").agg(sum("paid_use").alias("paid_use"), sum("free_use").alias("free_use"), sum("wsus_cnt").alias("wsus_cnt")) \
    .withColumn("paid_user", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("total_pu"), countDistinct("paid_user").alias("paid_pu"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("wsus_cnt").alias("wsus_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,total_pu,paid_pu,paid_use,total_use,wsus_cnt
0,A,costume,23781,13276,11444110,19414590,NaN
1,A,workshop,35487,19854,120209150,146114070,NaN
2,A,wsus,9521,4811,36161120,42785000,234807.0
3,B,costume,17807,12943,15771665,21410610,NaN
4,B,workshop,23759,16413,169083220,196482490,NaN
5,B,wsus,8372,5434,56827320,65210000,359766.0
6,C,costume,1059,961,2499080,2934710,NaN
7,C,workshop,1126,1012,30314900,34691600,NaN
8,C,wsus,492,439,12060340,13623200,75504.0
9,D,costume,64,60,233930,277520,NaN


In [91]:
# group별 G-COIN 사용 유저 수
load_data_mart("pc", ws_start_date, ws_day_30, "gcoin_use") \
    .join(only_costume_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop") \
                .when((col("sub_category").isin(["costume", "gear", "vehicle"])) | \
                          ((col("sub_category") == "crate") & (~col("event_name").like("%wsus%")) & (~col("event_name").like("%workshop%")) & (~col("event_name").like("%survivorpass%"))), "costume") \
                .when(col("event_name").like("%wsus%"), "wsus")) \
    .where(col("type").isin(["workshop", "costume", "wsus"])) \
    .groupBy("group").agg(countDistinct("account_id").alias("pu"), sum("paid_use").alias("paid_use"), sum(col("free_use") + col("paid_use")).alias("total_use")).orderBy("group").toPandas()

,group,pu,paid_use,total_use
0,A,52149,167814380,208313660
1,B,35195,241682205,283103100
2,C,1455,44874320,51249510
3,D,82,3642780,4147730
4,E,1,321480,359940


## b 그룹

In [47]:
costume_and_wsus_gcoin.select("event_name").distinct().show(truncate=False)

+-----------------------+
|event_name             |
+-----------------------+
|202203_wsus            |
|202206_wsus_progressive|
|202204_wsus_progressive|
|202205_wsus            |
+-----------------------+



In [24]:
from pyspark.sql.types import * 

def wsus_cnt(price):
    if price == 1800:
        return 10
    elif price == 200:
        return 1
    else:
        return 0
    
assign_wsus_cnt = udf(wsus_cnt, IntegerType())
costume_and_wsus_gcoin = costume_and_wsus_gcoin.withColumn("wsus_cnt", assign_wsus_cnt("price"))

In [50]:
costume_and_wsus_gcoin.groupBy("wsus_cnt").agg(count("*")).show(truncate=False)

+--------+--------+
|wsus_cnt|count(1)|
+--------+--------+
|1       |348590  |
|10      |234416  |
+--------+--------+



In [25]:
costume_and_wsus_gcoin_by_user = costume_and_wsus_gcoin.groupBy("account_id") \
    .agg(sum("wsus_cnt").alias("wsus_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))

In [26]:
costume_and_wsus_gcoin_by_user = costume_and_wsus_gcoin_by_user.withColumn("group", \
                                                                when(col("wsus_cnt") <=3, "A") \
                                                                .when((col("wsus_cnt") >= 4) & (col("wsus_cnt") <= 15), "B") \
                                                                .when((col("wsus_cnt") >= 16) & (col("wsus_cnt") <= 30), "C") \
                                                                .when((col("wsus_cnt") >= 31) & (col("wsus_cnt") <= 240), "D") \
                                                                .when((col("wsus_cnt") >= 241), "E") \
                                                                          )
costume_and_wsus_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_use").alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_use,paid_use,wsus_cnt
0,A,54821,16925400,2594885,84627
1,B,24239,35307000,16128980,185174
2,C,6932,28905800,23283455,156798
3,D,12614,193803800,171130575,1066128
4,E,2313,216724800,194283425,1200023


In [85]:
costume_and_wsus_gcoin_by_user.where(col("paid_use")>0).groupBy("group").agg(countDistinct("account_id").alias("paid_pu")).orderBy("group").toPandas()

,group,paid_pu
0,A,12490
1,B,13863
2,C,6735
3,D,12510
4,E,2309


In [27]:
costume_and_wsus_gcoin_by_user.toPandas().to_csv("./costume_and_wsus_gcoin_by_user.csv", index=False)

In [48]:
costume_and_wsus_gcoin_by_user = pd.read_csv("./costume_and_wsus_gcoin_by_user.csv")

In [80]:
# AU by group
user.withColumnRenamed("AccountId", "account_id").where(col("lastlogindate") >= ws_start_date).join(costume_and_wsus_gcoin_by_user, "account_id").groupBy("group").agg(countDistinct("account_id").alias("au")).orderBy("group").toPandas()

,group,au
0,A,40739
1,B,19505
2,C,6255
3,D,11617
4,E,2230


In [49]:
costume_and_wsus_gcoin_by_user = spark.createDataFrame(costume_and_wsus_gcoin_by_user)
workshop_b = ws_user.join(costume_and_wsus_gcoin_by_user.withColumnRenamed("account_id", "AccountId"), "AccountId")
workshop_b.groupBy("group", "action").agg(countDistinct("AccountId").alias("user_cnt")).orderBy("group").toPandas()

,group,action,user_cnt
0,A,craft,21413
1,A,open,29933
2,A,special_craft,8504
3,A,repurpose,7375
4,A,disassemble,20867
5,A,bp_workshop,25274
6,A,gcoin,15713
7,B,open,15871
8,B,gcoin,9689
9,B,disassemble,11902


In [50]:
workshop_b.groupBy("group").agg(countDistinct("AccountId").alias("ws user cnt")).orderBy("group").toPandas()

,group,ws user cnt
0,A,30773
1,B,16151
2,C,5660
3,D,10894
4,E,2158


In [28]:
pre_gcoin_by_type.join(costume_and_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .groupBy("account_id", "group", "type").agg(sum("paid_use").alias("paid_use"), sum("free_use").alias("free_use"), sum("wsus_cnt").alias("wsus_cnt")) \
    .withColumn("paid_user", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("total_pu"), countDistinct("paid_user").alias("paid_pu"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("wsus_cnt").alias("wsus_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,total_pu,paid_pu,paid_use,total_use,wsus_cnt
0,A,costume,54821,17219,20583150,61400590,NaN
1,A,wsus,54821,12490,2594885,16925400,84627.0
2,B,costume,24239,13488,21082390,34328130,NaN
3,B,wsus,24239,13863,16128980,35307000,185174.0
4,C,costume,6932,5949,11089325,13963840,NaN
5,C,wsus,6932,6735,23283455,28905800,156798.0
6,D,costume,12614,11108,24064830,29857530,NaN
7,D,wsus,12614,12510,171130575,193803800,1066128.0
8,E,costume,2313,2007,5667650,7124830,NaN
9,E,wsus,2313,2309,194283425,216724800,1200023.0


In [65]:
group_b_workshop_gcoin_by_product = ws_gcoin.join(costume_and_wsus_gcoin_by_user.select("account_id", "group"), "account_id").groupBy("group", "product_name") \
    .agg(countDistinct("account_id").alias("pu"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).toPandas()
group_b_workshop_gcoin_by_product.to_csv("./group_b_workshop_gcoin_by_product.csv", index=False)

In [83]:
group_b_workshop_gcoin_by_product.pivot(index="product_name", columns="group", values="pu")

group,A,B,C,D,E
product_name,,,,,
ARCHIVIST'S CHEST,3803,2602,1109,2359,484
ARCHIVIST'S CHEST BUNDLE (x1),8807,6032,2694,5594,1163
ARCHIVIST'S CHEST BUNDLE (x10),4323,4153,2358,6053,1516
ARCHIVIST'S CHEST BUNDLE (x5),3732,3249,1736,4050,935
HUNTER'S CHEST,6470,3814,1470,2967,578
HUNTER'S CHEST PACK (x1),4252,2955,1307,2922,673
HUNTER'S CHEST PACK (x5),2700,2243,1198,3166,870
KEY,9526,6208,2707,6127,1429


In [71]:
group_b_workshop_gcoin_by_product.sort_values(["group", "product_name"])

,group,product_name,pu,paid_use,total_use
30,A,ARCHIVIST'S CHEST,3803,711990,1378800
5,A,ARCHIVIST'S CHEST BUNDLE (x1),8807,6211260,9840240
27,A,ARCHIVIST'S CHEST BUNDLE (x10),4323,48798320,50613600
25,A,ARCHIVIST'S CHEST BUNDLE (x5),3732,7366040,9373200
31,A,HUNTER'S CHEST,6470,659040,1657320
32,A,HUNTER'S CHEST PACK (x1),4252,2235770,3602590
7,A,HUNTER'S CHEST PACK (x5),2700,19317435,21006400
11,A,KEY,9526,3389470,6076400
3,B,ARCHIVIST'S CHEST,2602,682710,1127880
16,B,ARCHIVIST'S CHEST BUNDLE (x1),6032,5405740,8098800


In [29]:
gcoin_by_type.join(costume_and_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .groupBy("account_id", "group", "type").agg(sum("paid_use").alias("paid_use"), sum("free_use").alias("free_use"), sum("wsus_cnt").alias("wsus_cnt")) \
    .withColumn("paid_user", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("total_pu"), countDistinct("paid_user").alias("paid_pu"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("wsus_cnt").alias("wsus_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,total_pu,paid_pu,paid_use,total_use,wsus_cnt
0,A,costume,8274,6087,6313080,8344000,NaN
1,A,workshop,15713,10507,88689325,103548550,NaN
2,A,wsus,5923,4016,26882885,31084000,170145.0
3,B,costume,5418,4491,5486100,6694970,NaN
4,B,workshop,9689,7879,91098835,104049070,NaN
5,B,wsus,5280,4356,36283975,41336800,226767.0
6,C,costume,2421,2126,3006660,3608210,NaN
7,C,workshop,4056,3564,57982455,65791130,NaN
8,C,wsus,2767,2455,23281240,26517400,145801.0
9,D,costume,5188,4606,8136350,9587700,NaN


In [92]:
# group별 G-COIN 사용 유저 수
load_data_mart("pc", ws_start_date, ws_day_30, "gcoin_use") \
    .join(costume_and_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop") \
                .when((col("sub_category").isin(["costume", "gear", "vehicle"])) | \
                          ((col("sub_category") == "crate") & (~col("event_name").like("%wsus%")) & (~col("event_name").like("%workshop%")) & (~col("event_name").like("%survivorpass%"))), "costume") \
                .when(col("event_name").like("%wsus%"), "wsus")) \
    .where(col("type").isin(["workshop", "costume", "wsus"])) \
    .groupBy("group").agg(countDistinct("account_id").alias("pu"), sum("paid_use").alias("paid_use"), sum(col("free_use") + col("paid_use")).alias("total_use")).orderBy("group").toPandas()

,group,pu,paid_use,total_use
0,A,19901,121885290,142976550
1,B,12080,132868910,152080840
2,C,4931,84270355,95916740
3,D,9936,303949060,343958750
4,E,2031,134989380,151862980


## c 그룹

In [30]:
only_wsus_gcoin = only_wsus_gcoin.withColumn("wsus_cnt", assign_wsus_cnt("price"))

In [31]:
only_wsus_gcoin_by_user = only_wsus_gcoin.groupBy("account_id") \
    .agg(sum("wsus_cnt").alias("wsus_cnt"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"))

In [32]:
only_wsus_gcoin_by_user = only_wsus_gcoin_by_user.withColumn("group", \
                                                when(col("wsus_cnt") <=10, "A") \
                                                .when((col("wsus_cnt") >= 11) & (col("wsus_cnt") <= 50), "B") \
                                                .when((col("wsus_cnt") >= 51) & (col("wsus_cnt") <= 100), "C") \
                                                .when((col("wsus_cnt") >= 101) & (col("wsus_cnt") <= 800), "D") \
                                                .when((col("wsus_cnt") >= 801), "E") \
                                                          )
only_wsus_gcoin_by_user.groupBy("group").agg(countDistinct("account_id").alias("user_cnt"), sum("total_use").alias("total_use"), sum("paid_use").alias("paid_use"), sum("wsus_cnt").alias("wsus_cnt")).orderBy("group").toPandas()

,group,user_cnt,total_use,paid_use,wsus_cnt
0,A,441496,299761000,22528835,1557540
1,B,34190,134708600,98032400,730118
2,C,7445,93058200,81686040,511904
3,D,5991,232449600,207615730,1285031
4,E,151,27655600,24780460,153394


In [33]:
only_wsus_gcoin_by_user.where(col("paid_use")>0).groupBy("group").agg(countDistinct("account_id").alias("paid_pu")).orderBy("group").toPandas()

,group,paid_pu
0,A,63196
1,B,29513
2,C,7362
3,D,5968
4,E,151


In [34]:
only_wsus_gcoin_by_user.toPandas().to_csv("./only_wsus_gcoin_by_user.csv", index=False)

In [57]:
only_wsus_gcoin_by_user = pd.read_csv("./only_wsus_gcoin_by_user.csv")

In [81]:
# AU by group
user.withColumnRenamed("AccountId", "account_id").where(col("lastlogindate") >= ws_start_date).join(only_wsus_gcoin_by_user, "account_id").groupBy("group").agg(countDistinct("account_id").alias("au")).orderBy("group").toPandas()

,group,au
0,A,284443
1,B,29372
2,C,6865
3,D,5569
4,E,147


In [58]:
only_wsus_gcoin_by_user = spark.createDataFrame(only_wsus_gcoin_by_user)
workshop_c = ws_user.join(only_wsus_gcoin_by_user.withColumnRenamed("account_id", "AccountId"), "AccountId")
workshop_c.groupBy("group", "action").agg(countDistinct("AccountId").alias("user_cnt")).orderBy("group").toPandas()

,group,action,user_cnt
0,A,craft,110217
1,A,special_craft,30836
2,A,repurpose,27355
3,A,bp_workshop,136934
4,A,disassemble,104178
5,A,open,179517
6,B,open,24966
7,B,repurpose,7709
8,B,craft,18439
9,B,special_craft,7953


In [87]:
workshop_c.select("action").distinct().show(truncate=False)

+-------------+
|action       |
+-------------+
|disassemble  |
|repurpose    |
|bp_workshop  |
|craft        |
|gcoin        |
|special_craft|
|open         |
+-------------+



In [59]:
workshop_c.groupBy("group").agg(countDistinct("AccountId").alias("ws user cnt")).orderBy("group").toPandas()

,group,ws user cnt
0,A,185918
1,B,25345
2,C,6220
3,D,5169
4,E,132


In [35]:
pre_gcoin_by_type.join(only_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .groupBy("account_id", "group", "type").agg(sum("paid_use").alias("paid_use"), sum("free_use").alias("free_use"), sum("wsus_cnt").alias("wsus_cnt")) \
    .withColumn("paid_user", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("total_pu"), countDistinct("paid_user").alias("paid_pu"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("wsus_cnt").alias("wsus_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,total_pu,paid_pu,paid_use,total_use,wsus_cnt
0,A,wsus,441496,63196,22528835,299761000,1557540
1,B,wsus,34190,29513,98032400,134708600,730118
2,C,wsus,7445,7362,81686040,93058200,511904
3,D,wsus,5991,5968,207615730,232449600,1285031
4,E,wsus,151,151,24780460,27655600,153394


In [66]:
group_c_workshop_gcoin_by_product = ws_gcoin.join(only_wsus_gcoin_by_user.select("account_id", "group"), "account_id").groupBy("group", "product_name") \
    .agg(countDistinct("account_id").alias("pu"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).toPandas()
group_c_workshop_gcoin_by_product.to_csv("./group_c_workshop_gcoin_by_product.csv", index=False)

In [84]:
group_c_workshop_gcoin_by_product.pivot(index="product_name", columns="group", values="pu")

group,A,B,C,D,E
product_name,,,,,
ARCHIVIST'S CHEST,12798,3220,918,801,17
ARCHIVIST'S CHEST BUNDLE (x1),30998,8103,2320,2029,46
ARCHIVIST'S CHEST BUNDLE (x10),11469,5868,2188,2214,66
ARCHIVIST'S CHEST BUNDLE (x5),9989,4207,1390,1401,39
HUNTER'S CHEST,22935,4689,1183,936,18
HUNTER'S CHEST PACK (x1),12043,3320,987,922,20
HUNTER'S CHEST PACK (x5),6433,2645,983,1026,26
KEY,36364,8074,2319,2215,60


In [72]:
group_c_workshop_gcoin_by_product.sort_values(["group", "product_name"])

,group,product_name,pu,paid_use,total_use
30,A,ARCHIVIST'S CHEST,12798,1819910,4199040
5,A,ARCHIVIST'S CHEST BUNDLE (x1),30998,13359680,24815760
27,A,ARCHIVIST'S CHEST BUNDLE (x10),11469,102828645,106996800
25,A,ARCHIVIST'S CHEST BUNDLE (x5),9989,15638440,20886000
31,A,HUNTER'S CHEST,22935,1436865,4876800
32,A,HUNTER'S CHEST PACK (x1),12043,4264330,7562570
7,A,HUNTER'S CHEST PACK (x5),6433,35615375,39233100
11,A,KEY,36364,6772645,13874600
3,B,ARCHIVIST'S CHEST,3220,695920,1243260
16,B,ARCHIVIST'S CHEST BUNDLE (x1),8103,5520480,8779920


In [36]:
gcoin_by_type.join(only_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .groupBy("account_id", "group", "type").agg(sum("paid_use").alias("paid_use"), sum("free_use").alias("free_use"), sum("wsus_cnt").alias("wsus_cnt")) \
    .withColumn("paid_user", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None))) \
    .groupBy("group", "type").agg(countDistinct("account_id").alias("total_pu"), countDistinct("paid_user").alias("paid_pu"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("wsus_cnt").alias("wsus_cnt")) \
    .orderBy("group", "type").toPandas()

,group,type,total_pu,paid_pu,paid_use,total_use,wsus_cnt
0,A,costume,29483,17550,15323030,23832340,NaN
1,A,workshop,67530,33428,181735890,222444670,NaN
2,A,wsus,38860,18492,93288240,115375800,627414.0
3,B,costume,6526,5231,5239870,6644250,NaN
4,B,workshop,13991,10828,95867340,110104130,NaN
5,B,wsus,10596,8961,69565680,80061400,438576.0
6,C,costume,1991,1657,2069600,2602830,NaN
7,C,workshop,3900,3172,44987450,51490280,NaN
8,C,wsus,3278,2938,40498005,46358800,255512.0
9,D,costume,1841,1512,2091190,2595700,NaN


In [93]:
# group별 G-COIN 사용 유저 수
load_data_mart("pc", ws_start_date, ws_day_30, "gcoin_use") \
    .join(only_wsus_gcoin_by_user.select("account_id", "group"), "account_id") \
    .withColumn("type", when(col("event_name").like("%workshop%"), "workshop") \
                .when((col("sub_category").isin(["costume", "gear", "vehicle"])) | \
                          ((col("sub_category") == "crate") & (~col("event_name").like("%wsus%")) & (~col("event_name").like("%workshop%")) & (~col("event_name").like("%survivorpass%"))), "costume") \
                .when(col("event_name").like("%wsus%"), "wsus")) \
    .where(col("type").isin(["workshop", "costume", "wsus"])) \
    .groupBy("group").agg(countDistinct("account_id").alias("pu"), sum("paid_use").alias("paid_use"), sum(col("free_use") + col("paid_use")).alias("total_use")).orderBy("group").toPandas()

,group,pu,paid_use,total_use
0,A,99365,290347160,361652810
1,B,18911,170672890,196809780
2,C,5132,87555055,100451910
3,D,4452,123552980,139453080
4,E,118,5819290,6534300
